In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Baixar o tokenizer do NLTK
nltk.download('punkt')

# Exemplo de dataset com descrições de produtos
data = {
    'produto': [
        "Smartphone iPhone 12 da Apple com 64GB de armazenamento.",
        "Notebook Dell Inspiron com 16GB de RAM e 1TB de SSD.",
        "Smartphone Samsung Galaxy S21 com câmera de 64MP.",
        "Televisão LG 55 polegadas 4K Ultra HD.",
        "Notebook Apple MacBook Air com M1 Chip.",
        "Câmera Canon EOS com lente de 24-70mm.",
        "Fone de ouvido Bluetooth Sony WH-1000XM4.",
        "Smartphone Xiaomi Mi 11 com Snapdragon 888.",
        "Tablet Samsung Galaxy Tab S6 com 128GB de armazenamento.",
        "Televisão Sony 65 polegadas 4K OLED."
    ]
}

df = pd.DataFrame(data)

# Tokenizar as descrições dos produtos
df['tokens'] = df['produto'].apply(lambda x: word_tokenize(x.lower()))

# Treinar o modelo Word2Vec nas descrições dos produtos
w2v_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Função para converter uma descrição de produto em um vetor médio de Word2Vec
def get_average_vector(tokens_list, model):
    vectors = [model.wv[word] for word in tokens_list if word in model.wv]
    if len(vectors) > 0:
        return sum(vectors) / len(vectors)
    else:
        return [0] * model.vector_size

# Criar vetores médios para os produtos
df['vector'] = df['tokens'].apply(lambda x: get_average_vector(x, w2v_model))

# Função para recomendar produtos com base em similaridade de cosseno
def recomendar_produtos(produto_descricao, df, model, top_n=3):
    # Pré-processar e converter o produto em um vetor
    tokens = word_tokenize(produto_descricao.lower())
    produto_vector = get_average_vector(tokens, model)
    
    # Calcular a similaridade de cosseno entre o produto e todos os outros produtos
    df['similarity'] = df['vector'].apply(lambda x: cosine_similarity([produto_vector], [x])[0][0])
    
    # Ordenar os produtos pela similaridade
    produtos_similares = df.sort_values(by='similarity', ascending=False).head(top_n)
    
    # Retornar os produtos mais similares
    return produtos_similares[['produto', 'similarity']]

# Exemplo de uso: recomendar produtos semelhantes a um novo produto
novo_produto = "Smartphone da Apple com 128GB de armazenamento."
produtos_recomendados = recomendar_produtos(novo_produto, df, w2v_model, top_n=3)

# Exibir as recomendações
print(f"Produtos recomendados para '{novo_produto}':\n")
print(produtos_recomendados)


Produtos recomendados para 'Smartphone da Apple com 128GB de armazenamento.':

                                             produto  similarity
0  Smartphone iPhone 12 da Apple com 64GB de arma...    0.825904
2  Smartphone Samsung Galaxy S21 com câmera de 64MP.    0.508201
8  Tablet Samsung Galaxy Tab S6 com 128GB de arma...    0.504061


[nltk_data] Downloading package punkt to /home/alexmarino/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
# Exemplo de uso: recomendar produtos semelhantes a um novo produto
novo_produto = "Smartphone Xiaomi"
produtos_recomendados = recomendar_produtos(novo_produto, df, w2v_model, top_n=5)

# Exibir as recomendações
print(f"Produtos recomendados para '{novo_produto}':\n")
print(produtos_recomendados)


Produtos recomendados para 'Smartphone Xiaomi':

                                             produto  similarity
7        Smartphone Xiaomi Mi 11 com Snapdragon 888.    0.422898
2  Smartphone Samsung Galaxy S21 com câmera de 64MP.    0.255538
0  Smartphone iPhone 12 da Apple com 64GB de arma...    0.170370
6          Fone de ouvido Bluetooth Sony WH-1000XM4.    0.053869
5             Câmera Canon EOS com lente de 24-70mm.    0.018591
